
<font color="#1874CD"><h1 align="left">**Datamecum**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Árboles de decisión**</h2></font>
<font color="#6E6E6E"><h3 align="left">Notebook: aplicación de las SVM a un problema de regresión</h3></font>
***

El objetivo de este *notebook* es familiarizarnos con el uso de las SVM en un problema de regresión. Concretamente, aprenderemos cómo se comportan los diferentes tipos de *kernel* usando problemas fáciles de interpretar y visualizar.

In [1]:
## ---------------------------------- ##
## Librerías 
## ---------------------------------- ##

import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

np.random.seed(19)

%matplotlib inline

Ahora, vamos a definir algunas funciones que nos serán de ayuda. La primera, *createDataSet_1D_regression*, se usará para generar problemas sintéticos de regresión. Los argumentos de esta función son:

* n: número de puntos.
* problem: tipo de problema. Puede ser 'linear', 'square' o 'sine'.
* noise_level: nivel de ruido

In [2]:
def createDataSet_1D_regression(n,problem,noise_level):
    
    x = np.random.rand(n,1)*2.0*np.pi
    noise = noise_level*np.random.randn(n,1)
    
    if problem == 'sine':
        y = 1.0+2.0*np.sin(x)
    elif problem == 'linear':
        y = 1.0+2.0*x
    elif problem == 'square':
        y = 1.0+2.0*x*x
    else:
        y = x
    
    y = y + noise    
    
    return x, y

La siguiente función, *plotData*, se usa para mostrar visualmente el dataset. Los argumentos son:

* x: coordenada OX de los puntos a mostrar.
* y: coordenada OY de los puntos a mostrar.
* style: estilo de los puntos.
* title: título de la gráfica.

In [3]:
def plotData(x,y,style,title):
    
    plt.plot(x,y,style)
    plt.grid(True)
    plt.axis([x.min()-0.2, x.max()+0.2, y.min()-0.2, y.max()+0.2])
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title(title)

Por último, definimos la función *plot_SVR_model* que dibuja el modelo de regresión SVR en el **espacio original de atributos**. Sus argumentos son:

* xtrain: valores X en train.
* ytrain: variable predictora en train.
* xtest: valores X en test.
* ytest: variable predictora en test.
* svr: SVM regresión.
* title: título de la gráfica.

In [4]:
def plot_SVR_model(xtrain,ytrain,xtest,ytest,svr,title):
    
    x_min = np.min([xtrain.min(), xtest.min()]) - .2
    x_max = np.min([xtrain.max(), xtest.max()]) + .2
    y_min = np.min([ytrain.min(), ytest.min()]) - .2
    y_max = np.min([ytrain.max(), ytest.max()]) + .2
    
    plt.figure(figsize=(5,5))

    font_size = 14    
    ax1 = plt.subplot(1,1,1)
    aux = np.linspace(x_min, x_max, 101)
    xx = aux.reshape(np.shape(aux)[0],1)
    yy = svr.predict(xx)
    plt.plot(xx,yy,'-b',ms=10)
    plt.plot(xtrain,ytrain,'.b')
    for i in svr.support_:
        plt.plot(xtrain[i],ytrain[i],'ro')

    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title(title)
    
    incy = 0.05*(y_max-y_min)
    lims = [y_min-incy, y_max+incy]
        
    plt.figure(figsize=(10,5))
    ax2 = plt.subplot(1,2,1)
    ytrain_predicted = svr.predict(xtrain)
    plt.plot(ytrain, ytrain_predicted, 'b.', zorder=1)
    plt.plot(lims, lims, 'k-', zorder=1)
    ax2.set_xlim(lims)
    ax2.set_ylim(lims)
    plt.xlabel('real y', fontsize=font_size)
    plt.ylabel('predicted y', fontsize=font_size)
    plt.title('Training set', fontsize=font_size)
    plt.grid(True)
    
    ax3 = plt.subplot(1,2,2)
    ytest_predicted = svr.predict(xtest)
    plt.plot(ytest, ytest_predicted, 'b.', zorder=1)
    plt.plot(lims, lims, 'k-', zorder=1)
    ax3.set_xlim(lims)
    ax3.set_ylim(lims)
    plt.xlabel('real y', fontsize=font_size)
    plt.ylabel('predicted y', fontsize=font_size)
    plt.title('Test set', fontsize=font_size)
    plt.grid(True)

    plt.tight_layout()
    plt.show()

Una vez definidas las funciones, generamos problemas con dos conjuntos de datos (entrenamiento y test).

In [5]:
## Inputs
n = 200
problem = 'sine'
noise_level = 0.2

In [6]:
## Creamos el dataset
X, clase = createDataSet_1D_regression(n, problem, noise_level)

In [7]:
## Dividimos en entrenamiento y test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, clase, test_size=0.3, random_state=3)

In [ ]:
## Visualizamos

plt.figure(figsize=(10,4))
plt.subplot(121)
plotData(X_train, y_train, 'o', "Training data")
plt.subplot(122)
plotData(X_test, y_test, 'o', "Test data")

Con los datos ya generados, entrenamos un <a href="https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html"> SVR </a>.

El constructor SVR requiere los siguientes argumentos:

* **Tipo de kernel.** Puede ser 'poly', 'linear', 'rbf', ...
* **Parámetro gamma.** Solo para kernels RBF.
* **Parámetro epsilon.** Margen de error admisible.
* **Parámetro de regularización C.** Cuanto más elevado, tendremos modelos con tasa de error más baja pero más complejos.
* **Grado.** Para kernel polinómico.
* **Coeficinete de orden 0.** Para kernel polinómico.

In [ ]:
## Construcción del modelo de regresión:
svr = SVR(kernel='rbf', epsilon=1, C=1)
svr.fit(X_train, np.ravel(y_train))

print(svr)

## Gráfica
plot_SVR_model(X_train, 
               y_train, 
               X_test, 
               y_test, 
               svr, 
               "Training. Rojo: vectores de soporte")

## Cálculo del acierto (RMSE y R squared) en los conjuntos de entrenamiento y test
print("RMSE in training: %.2f"
      % np.sqrt(np.mean((svr.predict(X_train) - y_train) ** 2)))
print("RMSE in test: %.2f"
      % np.sqrt(np.mean((svr.predict(X_test) - y_test) ** 2)))

score_train = svr.score(X_train, y_train)
print("Explained variance score in training (1: perfect): %f" % (score_train))
score_test = svr.score(X_test, y_test)
print("Explained variance score in test (1: perfect): %f" % (score_test))

Ahora, buscamos los mejores hiperparámetros usando el mismo problema sintético.

In [ ]:
## Mismo que antes pero con búsqueda de los parámetros óptimos para el modelo SVR

# Construcción del modelo:
par_grid = [
  {
      'C': [1, 10, 100, 1000],
      'epsilon': [1.0, 0.1, 0.01],
      'kernel': ['poly', 'linear', 'rbf'],
      'degree': [2],
      'coef0': [1.0]
  },
 ]

## Comienzo grid search
svr_gridsearch = GridSearchCV(estimator=SVR(), param_grid=par_grid) ## Por defecto, cv = 'None' que se refiere a 5-fold
svr_gridsearch.fit(X_train, np.ravel(y_train))  

print(f"Mejor score (R2) del grid search: {svr_gridsearch.best_score_}")

## Guardo el mejor modelo encontrado
svr = svr_gridsearch.best_estimator_

## Vemos hiperparámetros
print(f"Parámetros: {svr.get_params}")

## Lo reentreno
svr.fit(X_train, np.ravel(y_train))

## R squared (coeficiente de determinación)
print(f"Score en train: {svr.score(X_train, np.ravel(y_train))}")

## Gráficas
plot_SVR_model(X_train, y_train, X_test, y_test, svr, "Training. Rojo: vectores de soporte")
print(svr)